# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [11]:
# Question 1: What is the sum of the outputs of the generator for limit = 5? 
generator = square_root_generator(limit)

limit = 5
generator = square_root_generator(limit)

print(sum(generator))

8.382332347441762


In [12]:
# Question 2: What is the 13th number yielded
limit = 13 
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [13]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [15]:
import dlt

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

generators_pipeline.run(people_1(),
						table_name="people",
						write_disposition="append")

generators_pipeline.run(people_2(),
						table_name="people",
						write_disposition="append");

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7f76a46858a0>, metrics={'1707268047.5892477': [{'started_at': DateTime(2024, 2, 7, 1, 7, 27, 951756, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 7, 1, 7, 28, 384384, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////workspaces/Zoomcamp2024/workshop-1-dlt/dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='generators', loads_ids=['1707268047.5892477'], load_packages=[LoadPackageInfo(load_id='1707268047.5892477', package_path='/home/codespace/.dlt/pipelines/dlt_ipykernel_launcher/load/loaded/1707268047.5892477', state='loaded', schema=Schema dlt_ipykernel_launcher at 140146600060496, schema_update={'people': {'name': 'people', 'columns': {'occupation': {'name': 'occupation', 'data_type': 'text', 'nullabl

In [17]:
# Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.
import duckdb 

# connect to db
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# set search path
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

# find the sum of ages
ages = conn.sql("SELECT SUM(Age) FROM people").df()
display(ages)

,sum(Age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [19]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

UsageError: Line magic function `%%capture` not found.


In [41]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

generators_pipeline.run(people_1(),
						table_name="people2",
						write_disposition="merge",
                        primary_key="ID")

generators_pipeline.run(people_2(),
						table_name="people2",
						write_disposition="merge",
                         primary_key="ID");

In [44]:
# Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

# connect to db
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# set search path
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

# find the sum of ages
ages = conn.sql("SELECT SUM(age) FROM people2").df()
display(ages)

,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX